In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline


In [2]:
df = pd.read_csv('D:\jupyter\projects\data\healthcare-dataset-stroke-data.csv')

In [3]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


---

#### Describe the numerical data ?

### Q: How many missing values are there in each column ?

In [4]:
print(df.isnull().sum())

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


### Define the X and y so that the X are the Features and y is the targets

In [10]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [21]:
X = df.drop('stroke', axis=1)
X.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked


In [22]:
y = df['stroke']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

### Split the data into Testing, Training for now ( validation set after confirmation of workflow)

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


### Logistic Regression Pipeline

In [50]:
column_trans = make_column_transformer(
    (OneHotEncoder(),['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']),
    remainder='passthrough')

In [51]:
imp = SimpleImputer(missing_values='NaN', strategy='mean')

In [52]:
logreg = LogisticRegression()

In [54]:
smk = SMOTETomek()

In [59]:
steps_logreg = [('Imputation', imp),
                ('scaler', StandardScaler(with_mean=True)),
                ('Column_Transformation', column_trans),
                ('Smote Tomek', smk),
                ('logistic_regression', logreg)]

pipeline_logreg = Pipeline(steps_logreg)

parameters = {
    'logistic_regression__solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'logistic_regression__C': [100, 10, 1.0],
    'logistic_regression__max_iter': [100000]
             }


X_trainn, X_test, y_trainn, y_test = train_test_split(X_smt, y_smt, test_size=0.2, random_state=42, stratify=y)

X_train, y_train = smk.fit_resample(X_trainn, y_trainn)

cv_logreg = GridSearchCV(pipeline_logreg, param_grid=parameters, cv=5)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTETomek()' (type <class 'imblearn.combine._smote_tomek.SMOTETomek'>) doesn't

In [ ]:
cv_logreg.fit(X_train, y_train)

In [ ]:
print(cv_logreg.best_params_)

In [ ]:
y_pred_logreg = cv_logreg.predict(X_test)

### Logistic Regression Classification report

In [ ]:
print(cv_svm.best_params_)

In [ ]:
y_pred_svm = cv_svm.predict(X_test)

In [ ]:
y_pred_svm

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_svm))

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Junaid's Comments:

So to be explicit, the process would be:

1. Split data into training, validation and test
2. preprocess the data (all splits)
3. If there is a class imbalance, perform up sampling or downsampling (usually via the SMOTE method) on the training data ONLY
4. fit model on training data
5. Perform hyper parameter tuning using the validation set as checks to see if model performance is improving - be careful to avoid overfitting
6. Once you have a final model you’re happy with, compute its adequacy metrics on the test data set

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)